### План работы:
1. Дернуть данные прям из табличке excel, потом обработать все нужные показатели, которые я раньше писал; +
2. Создать таблицу в базе данных; +
3. Из всех задач построить dag; +
4. Подключить все это к airflow; +
5. Проверить что дашик видит все эти данные; +
6. Построить дашборд.+

In [1]:
from datetime import datetime, timedelta
import pandas as pd
import seaborn as sns
from airflow.decorators import dag, task
from clickhouse_driver import Client
from airflow.models import Variable

sns.set(rc={"figure.figsize": (12, 8)})

In [2]:
connection_employment = Client(host='localhost')

In [3]:
def load_prep():
    #Сначала просто загрузим даг
    sheet_id = "1WSxfMdjzHBaIa7AzKzjOL5gQCz595ytpUAPmbBvrGOg"
    df = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv")
    #Далее переименую и дропну ненужные мне колонки
    df.rename(columns={"Дата старта": "Start_date", "Дата оффера": "Contract_date", "ФИО": "Full_name", "Пол": "Gender",
                       "Возраст": "Age", "Старт/смена": "Start_Shift", "Текущий статус": "Current_situation",
                       "Контакты": "Contacts", "Почта": "Mail", "Город": "City_Country", "Курс": "Courses",
                       "Профессия": "Grade", "Статус": "Status", "Пришел из сферы": "Previous_experience",
                       "Последнее место работы ": "Last_job", "Трудоустроен в": "Current_Job",
                       "номер потока": "Class_number", "дата старта потока": "Start_Class_date",
                       "логин студента": "student_login", "Кто ведет": "Student_hr", "Unnamed: 0": "Student_id"},
              inplace=True)
    df.drop(
        ["Ссылка на папку студента", "Ссылка на резюме", "Дополнительно", "Комментарий после карьерной консультации",
         "Current_situation", "Previous_experience"], axis=1, inplace=True)
    #Далее переведу формат дат в нормальный вид
    df[["Start_date", "Contract_date", "Start_Class_date"]] = df[
        ["Start_date", "Contract_date", "Start_Class_date"]].apply(lambda x: pd.to_datetime(x, dayfirst=True))
    return df

In [4]:
df = load_prep()
df

,Student_id,Start_date,Contract_date,Full_name,Gender,Age,Start_Shift,Contacts,Mail,City_Country,Courses,Grade,Status,Current_Job,Start_Class_date,Class_number,student_login,Student_hr,Last_job
0,1.0,2021-11-01,2021-11-18,Маргарита Мариева,Ж,25.0,1.0,8-952-798-30-85,margarita.marieva@gmail.com,Saint Petersburg,Data analyst,junior,Нашел работу,Вкусвилл,NaT,5,NaN,NaN,ООО Технолюкс
1,2.0,2021-04-20,2021-05-11,Тихонова Марина,Ж,26.0,3.0,8-965-778-86-09,marinane7@gmail.com,Saint Petersburg,Data analyst,junior+,Нашел работу,Яндекс,NaT,NaN,NaN,NaN,Geekbrains
2,3.0,NaT,NaT,Трубникова Юлия,Ж,28.0,3.0,8-903-092-04-89,korovaevajulia@gmail.com,Saint Petersburg,Data analyst,middle,Не ищет,NaN,2020-10-01,5,NaN,NaN,PwC
3,4.0,NaT,NaT,Жарков Антон,М,32.0,2.0,@ZharkovAnton,19antoni88@gmail.com,Moscow,Data analyst,junior,Не ищет,NaN,NaT,NaN,NaN,NaN,CТБ Интегратор
4,5.0,2021-05-17,2021-06-28,Зарипов Айдар,М,27.0,3.0,8-911-039-74-37,zaripov0392@mail.ru,Saint Petersburg,Data analyst,middle,Нашел работу,Окко,2020-10-01,5,NaN,NaN,Civitta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,NaN,NaT,NaT,Блажеева Любовь,Ж,NaN,NaN,@lyublia,l.blazheeva@yandex.ru,NaN,Data analyst,junior,NaN,NaN,2022-05-15,24,l.blazheeva@yandex.ru,Белла,NaN
620,NaN,NaT,NaT,Моршнева Юлия,Ж,NaN,NaN,+79777106793,morshnevayv@gmail.com,NaN,Data analyst,junior,NaN,NaN,2022-05-16,24,morshnevayv@gmail.com,NaN,NaN
621,NaN,NaT,NaT,Якушев Сергей,М,43.0,1.0,@tezzz79,yakushev.mgn@gmail.com,Magnitogorsk,Data analyst,junior,В поиске,NaN,2022-09-09,28,s-jakushev-28,Белла,NaN
622,NaN,NaT,NaT,Макогон Сергей,М,NaN,NaN,999160374,makogon22@mail.ru,NaN,Data analyst,junior,NaN,NaN,2022-04-15,23,s-makogon,NaN,NaN


Снизу рассчитаем время между получением контракта и датой старта:

In [5]:
def jobs_score(df):
    #Сначала просто посчитаю разницу во времени получения контракта, а после на основе данной колонки создам еще одну
    df["time_difference"] = (df["Contract_date"] - df["Start_date"]).dt.days
    #Расчет показателя общего показателя success_rate
    lst_job_scores = []
    for i in df.time_difference:
        if i != "nan" and i != 0 and 0 < i <= 90:
            lst_job_scores.append("Success")
        elif i != "nan" and i != 0 and i > 90:
            lst_job_scores.append("Fail")
        else:
            lst_job_scores.append("Unknown")
    df["success_rate"] = lst_job_scores
    df.time_difference.fillna(0,
                              inplace=True)  #Ставлю значение 0, так как пока неизвестно устроен ли студент, или просто кто-то не поставил дату контракта :(
    df = df.query("time_difference >= 0")  #Проверка чтобы все точно нормально отработало
    return df

In [6]:
df = jobs_score(df)

Далее сделаю функцию для преобразования в возрастную группу:

In [7]:
def age(df):
    lst = []
    for i in df.Age:
        if 16 <= i <= 25:
            lst.append("first_group")
        elif 25 < i <= 35:
            lst.append("second_group")
        elif i > 35:
            lst.append("third_group")
        else:
            lst.append("Unknown")
    df["age_group"] = lst

    #Также в этом же таске изменим формат возраста, чтобы не пилить отдельный, ведь все остальные типы правильные:
    df["Age"].fillna(0, inplace=True)
    df["Age"] = df["Age"].astype("int32")

    return df

In [8]:
df = age(df)

/tmp/ipykernel_91194/1065393909.py:12 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/onixx/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:6392 SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_91194/1065393909.py:16 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

Немного поисследую данные до фильтра:

In [9]:
df.query("Start_date == 'NaT'")

,Student_id,Start_date,Contract_date,Full_name,Gender,Age,Start_Shift,Contacts,Mail,City_Country,...,Status,Current_Job,Start_Class_date,Class_number,student_login,Student_hr,Last_job,time_difference,success_rate,age_group
2,3.0,NaT,NaT,Трубникова Юлия,Ж,28,3.0,8-903-092-04-89,korovaevajulia@gmail.com,Saint Petersburg,...,Не ищет,NaN,2020-10-01,5,NaN,NaN,PwC,0.0,Unknown,second_group
3,4.0,NaT,NaT,Жарков Антон,М,32,2.0,@ZharkovAnton,19antoni88@gmail.com,Moscow,...,Не ищет,NaN,NaT,NaN,NaN,NaN,CТБ Интегратор,0.0,Unknown,second_group
9,10.0,NaT,NaT,Надежда Скворцова (Агеева),Ж,30,2.0,8-916-905-53-81,nadezda_sk@abbyy.com,Moscow,...,Не ищет,NaN,2021-09-21,16,NaN,NaN,NaN,0.0,Unknown,second_group
20,22.0,NaT,NaT,Быстрова Эмма,Ж,31,2.0,@emmaby,bystrova.emma@gmail.com,Moscow,...,Не ищет,NaN,2021-06-15,13,NaN,NaN,Онэлия,0.0,Unknown,second_group
28,30.0,NaT,NaT,NaN,Ж,30,3.0,@a_lena_titova,Elena.n.titova@yandex.ru,NaN,...,Не ищет,NaN,NaT,8,NaN,NaN,Россельхозбанк,0.0,Unknown,second_group
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,NaN,NaT,NaT,Блажеева Любовь,Ж,0,NaN,@lyublia,l.blazheeva@yandex.ru,NaN,...,NaN,NaN,2022-05-15,24,l.blazheeva@yandex.ru,NaN,NaN,0.0,Unknown,Unknown
620,NaN,NaT,NaT,Моршнева Юлия,Ж,0,NaN,+79777106793,morshnevayv@gmail.com,NaN,...,NaN,NaN,2022-05-16,24,morshnevayv@gmail.com,NaN,NaN,0.0,Unknown,Unknown
621,NaN,NaT,NaT,Якушев Сергей,М,0,NaN,@tezzz79,yakushev.mgn@gmail.com,NaN,...,NaN,NaN,2022-09-09,28,s-jakushev-28,NaN,NaN,0.0,Unknown,Unknown
622,NaN,NaT,NaT,Макогон Сергей,М,0,NaN,999160374,makogon22@mail.ru,NaN,...,NaN,NaN,2022-04-15,23,s-makogon,NaN,NaN,0.0,Unknown,Unknown


Напишу отдельный таск где я буду фильтровать дату:

In [10]:
def filter_data(df_final):
    #Меняю всем колонкам на всяких случай значения, чтобы не было проблем с типами данных:
    df_final[["Full_name", "Gender", "Contacts", "Mail", "City_Country", "Courses", "Grade", "Status", "Current_Job",
              "student_login", "Student_hr", "Last_job", "success_rate", "age_group"]] = df_final[
        ["Full_name", "Gender", "Contacts", "Mail", "City_Country", "Courses", "Grade", "Status", "Current_Job",
         "student_login", "Student_hr", "Last_job", "success_rate", "age_group"]].astype(str)
    df_final.student_login = df_final.student_login.replace("nan", "Unknown")
    df_final.Student_hr = df_final.Student_hr.replace("nan", "Unknown")

    #Здесь ставлю одинаковые значения, чтобы не было проблем с добавлением в бд:
    df_final["Contract_date"].fillna("2000-01-01", inplace=True)
    df_final["Start_date"].fillna("2000-01-01", inplace=True)
    df_final["Start_Class_date"].fillna("2000-01-01", inplace=True)

    #Остальные значения ставлю как Unknown:
    df_final.replace("nan", "Unknown", inplace=True)

    #Также ставлю значения, которых точно не будет в данных, фильтрую прям в superset:
    df_final.Start_Shift.fillna(0.0, inplace=True)
    df_final.Class_number.fillna(-5.0, inplace=True)

    df_final.City_Country = df_final.City_Country.replace("Saint Petersburg", "Saint-Petersburg")

    #Оставляем только те записи студентов, которые пришли к нам на трудоустройство:
    df_final = df_final.query("Start_date != '2000-01-01'")
    return df_final

### Условные значения, которые после будут помогать мне в процессе работы:

- Если колонка Contract_date == 2000-01-01 значит дата контракта неизвестна;
- Если колонка Start_date == 2000-01-01 значит дата старта неизвестна;
- Если колонка Start_Class_date == 2000-01-01 значит дата старта потока неизвестна;
- Если колонка time_difference равна 0, значит студент не получил офер.

Остальные фильтры также были в работе, но они не являются столь важными как эти.

In [11]:
df = filter_data(df)
df.head()

/home/onixx/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:3641 SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/onixx/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:5516 SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/onixx/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:6392 SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/onixx/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5238 SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

,Student_id,Start_date,Contract_date,Full_name,Gender,Age,Start_Shift,Contacts,Mail,City_Country,...,Status,Current_Job,Start_Class_date,Class_number,student_login,Student_hr,Last_job,time_difference,success_rate,age_group
0,1.0,2021-11-01,2021-11-18,Маргарита Мариева,Ж,25,1.0,8-952-798-30-85,margarita.marieva@gmail.com,Saint Petersburg,...,Нашел работу,Вкусвилл,2000-01-01,5,Unknown,Unknown,ООО Технолюкс,17.0,Success,first_group
1,2.0,2021-04-20,2021-05-11,Тихонова Марина,Ж,26,3.0,8-965-778-86-09,marinane7@gmail.com,Saint Petersburg,...,Нашел работу,Яндекс,2000-01-01,-5.0,Unknown,Unknown,Geekbrains,21.0,Success,second_group
4,5.0,2021-05-17,2021-06-28,Зарипов Айдар,М,27,3.0,8-911-039-74-37,zaripov0392@mail.ru,Saint Petersburg,...,Нашел работу,Окко,2020-10-01,5,Unknown,Unknown,Civitta,42.0,Success,second_group
5,6.0,2021-04-22,2021-05-11,Пономарев Валентин,М,24,1.0,8-981-955-87-61,ponvalentine@gmail.com,Saint Petersburg,...,Нашел работу,Karuna,2000-01-01,-5.0,Unknown,Unknown,Linxdatacenter,19.0,Success,first_group
6,7.0,2022-06-06,2022-07-12,Квашневский Олег,М,24,3.0,@biotvrg,oleg.kvashnevsky@mail.ru,Saint Petersburg,...,Нашел работу,Bnovo,2020-08-01,3,Unknown,Unknown,Essity,36.0,Success,first_group


##### Проверка данных, а также место для ответов на вопросы коллег:

In [12]:
df.query("Contract_date == '2000-01-01'")

,Student_id,Start_date,Contract_date,Full_name,Gender,Age,Start_Shift,Contacts,Mail,City_Country,...,Status,Current_Job,Start_Class_date,Class_number,student_login,Student_hr,Last_job,time_difference,success_rate,age_group
27,29.0,2021-08-23,2000-01-01,Трошин Александр,М,40,2.0,@Grodno_Raymond,Ray_Beam2@mail.ru,Belarus,...,В поиске,Unknown,2000-01-01,7.0,Unknown,Оксана,Яндекс,0.0,Unknown,third_group
57,59.0,2021-09-01,2000-01-01,Кулагин Владимир,М,37,2.0,@k17vv,k17vv@mail.ru,Krasnoyarsk,...,Пассивный поиск,Unknown,2000-01-01,10.0,Unknown,Оксана,Гк Пирамида,0.0,Unknown,third_group
58,60.0,2021-09-15,2000-01-01,Май Антон,М,29,2.0,@A_May_615,may.a.a@mail.ru,Novosibirsk,...,В поиске,Unknown,2021-02-15,9.0,Unknown,Белла,Media Buyer,0.0,Unknown,second_group
69,71.0,2022-09-05,2000-01-01,Анохина Инзира,Ж,32,2.0,Inzira90,Inzira90@yandex.ru,Moscow,...,Пассивный поиск,Unknown,2000-01-01,9.0,Unknown,Белла,Яндекс,0.0,Unknown,second_group
100,102.0,2021-11-01,2000-01-01,Александр Хилько,М,29,3.0,375293316446,hilkoa@mail.ru,Belarus,...,В поиске,Unknown,2000-01-01,1.0,Unknown,Оксана,ОМА,0.0,Unknown,second_group
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,537.0,2022-09-18,2000-01-01,Салехи Ника,Ж,33,2.0,@nika_salehi_pr,nika-az@mail.ru,Saint-Petersburg,...,В поиске,Unknown,2022-12-03,22.0,nika-az@mail.ru,Белла,Unknown,0.0,Unknown,second_group
538,540.0,2022-09-01,2000-01-01,Мангурсузян Рафаэль,М,35,2.0,https://t.me/ra_fa_el_0,mrvsea@gmail.com,Moscow,...,В поиске,Unknown,2021-11-01,18.0,r-mangursuzjan-18,Белла,Unknown,0.0,Unknown,second_group
549,551.0,2022-09-16,2000-01-01,Тимошенко Анна,Ж,37,3.0,https://t.me/aiotena,01anna.timoshenko@gmail.com,Vietnam,...,В поиске,Unknown,2022-08-19,10.0,01anna.timoshenko@gmail.com,Оксана,ЦПУР,0.0,Unknown,third_group
553,NaN,2022-09-20,2000-01-01,Матвей Майоров,М,24,3.0,https://t.me/matveymex,mmat9@me.com,Moscow,...,В поиске,Unknown,2022-05-01,7.0,m-majorov,Оксана,Otkrtitie Bank,0.0,Unknown,first_group


In [ ]:
df.at[58, "Full_name"] = "Апрель Антон"

Напишу таск, который будет очищать данные и после заново загружать, при эом сейчас выполню самый простой вариант, а далее буду его дорабатывать:

In [ ]:
def clean_upload_data(df):
    #Здесь будет очищать данные:
    connection_employment.execute("TRUNCATE default.employment_final")

    #Здесь будем добавлять данные:
    connection_employment.execute("INSERT INTO default.employment_final VALUES", df.to_dict(orient="records"))
    connection_employment.execute('OPTIMIZE TABLE default.employment_final DEDUPLICATE BY Student_id')

In [ ]:
clean_upload_data(df)

In [ ]:
df.Student_hr.unique()

In [ ]:
df.query("Student_hr == 'Настя'").Full_name.to_excel("your_studnets.xlsx", index=False)

In [ ]:
df.query("Student_hr == 'Белла'").Full_name.to_excel("your_studnets.xlsx", index=False)

Рассчитаем показатель успеваемости за последние 3 месяца:

In [ ]:
last_three_month = df.query("Start_date >= '2022-06-12'")

In [ ]:
round((last_three_month.query("time_difference <= 90 and time_difference != 0").shape[0] / last_three_month.shape[
    0]) * 100, 2)

Хочу посмотреть на распределение за сколько дней студенты находили работу:

In [ ]:
sns.histplot(last_three_month.time_difference);

In [ ]:
default_args = {
    'owner': 'v-saharov',
    'depends_on_past': False,
    'retries': 2,
    'retry_delay': timedelta(minutes=5),
    'start_date': datetime(2022, 4, 8),
}
schedule_interval = "* 22 * * 7"

In [ ]:
#Для clikchouse на сервере:
client = Client(host=Variable.get("CLICKHOUSE_HOST"),
                port=Variable.get("CLICKHOUSE_PORT"),
                user=Variable.get("CLICKHOUSE_USER"),
                password=Variable.get("CLICKHOUSE_PASSWORD"),
                database='analytics')